In [ ]:
# Load a ROMS grid
import os, sys
from gridtools.gridutils import GridUtils
from gridtools.grids import roms
from pyproj import Proj
import cartopy.crs as crs
import xarray as xr

# Set a place to write files
os.environ["PYROMS_GRIDID_FILE"] = "/import/AKWATERS/jrcermakiii/configs/Arctic6/roms/gridid.txt"
wrkDir = '/import/AKWATERS/jrcermakiii/configs/zOutput'
inputDir = os.path.join(wrkDir, 'INPUT')

grd = GridUtils()

romsObj = roms.ROMS()
romsGrd = romsObj.get_ROMS_grid('ARCTIC6')

In [ ]:
print(dir(romsGrd.hgrid))

In [ ]:
print(romsGrd.hgrid.mask_polygon)

In [ ]:
ds = xr.Dataset()
# Vertices of grid boxes
#ds['lat'] = (('ny', 'nx'), romsGrd.hgrid.lat_vert)
#ds['lon'] = (('ny', 'nx'), romsGrd.hgrid.lon_vert)

# Points
ds['lat'] = (('ny', 'nx'), romsGrd.hgrid.lat_rho)
ds['lon'] = (('ny', 'nx'), romsGrd.hgrid.lon_rho)

# Mask values
ds['mask'] = (('ny', 'nx'), romsGrd.hgrid.mask)

# Enable 2D mapping of coordinates
ds = ds.set_coords(['lat', 'lon'])

In [ ]:
# Informal mask editor 

# This does not change anything yet

import xarray as xr #version 0.15.1
import hvplot.xarray #version 0.6.0
import holoviews as hv
import geoviews as gv #version 1.8.1
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt
gv.extension('bokeh')

clickedValues = []
lastClickY = -1
lastClickX = -1
gridSubset = 50
lon = -1
lat = -1
xloc = -1
yloc = -1

kwargs = dict()
land_color = kwargs.pop('land_color', (0.6, 1.0, 0.6))
sea_color = kwargs.pop('sea_color', (0.6, 0.6, 1.0))
customCM = plt.matplotlib.colors.ListedColormap([land_color, sea_color],
                                                 name='land/sea')

def getGridSubset(lClickY, lClickX, grd):
    global gridSubset
    
    grdShape = grd.shape
    
    firstY = lClickY - gridSubset
    lastY = lClickY + gridSubset
    firstX = lClickX - gridSubset
    lastX = lClickX + gridSubset
    
    if firstY < 0:
        firstY = 0
        lastY = gridSubset - 1
        
    if firstX < 0:
        firstX = 0
        lastX = gridSubset - 1
    
    if lastY > grdShape[0]-1:
        firstY = grdShape[0] - gridSubset
        lastY = grdShape[0] - 1
    
    if lastX > grdShape[1]-1:
        firstX = grdShape[1] - gridSubset
        lastX = grdShape[1] - 1
    
    return [firstY, lastY, firstX, lastX]

    
# convert to numpy
def great_circle(lon1, lat1, lon2, lat2):
    # REF: https://medium.com/@petehouston/calculate-distance-of-two-locations-on-earth-using-python-1501b1944d97
    # Convert to numpy; map requires scalars
    # lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    dist = 6371 * (
        np.arccos(np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(lon1 - lon2))
    )
    
    return dist

def plotMap(x, y):
    
    global customCM, clickedValues, da, lastClickX, lastClickY, lon, lat, yloc, xloc
    
    lon = x
    lat = y
    
    # If undefined, set it to a center grid point
    if lastClickY == -1:
        lastClickY = int(da.shape[0] / 2)
    if lastClickX == -1:
        lastClickX = int(da.shape[1] / 2)

    if x is not None:
        
        # Find nearest grid point for curvilinear grid
        d = great_circle(lon, lat, da['lon'], da['lat'])
        ind = np.nonzero(np.equal(d, np.amin(d)))
        yloc = int(ind[0][0])
        xloc = int(ind[1][0])
        
        clickedValues.append([lon, lat, yloc, xloc])      
        da.loc[yloc, xloc] = np.where(da.loc[yloc, xloc] == 0, 1, 0)
        
        lastClickY = yloc
        lastClickX = xloc
        
    # Use subset method to edit grids
    [gy1, gy2, gx1, gx2] = getGridSubset(lastClickY, lastClickX, da)

    # This has a side effect of dynamically changing the colorbar and the
    # rendered image.
    
    # REF: https://hvplot.holoviz.org/user_guide/Geographic_Data.html#declaring-an-output-projection
    plt = da.sel(ny = slice(gy1, gy2), nx = slice(gx1, gx2)).hvplot.quadmesh(
        'lon', 'lat', 'mask', projection=ccrs.Orthographic(-160, 90),
        global_extent=True, frame_height=540, cmap=hv.plotting.util.process_cmap(customCM),
        coastline='10m', clim=(0,1)
    )
    opt_kwargs = dict()
    plt.opts(title='Ocean Mask Editor', **opt_kwargs)
    
    return plt

da = ds["mask"]

tap_stream = hv.streams.Tap()
dmap = gv.DynamicMap(plotMap, streams=[tap_stream])
dmap

In [ ]:
%pylab

PROJSTRING = "+proj=stere +lon_0=160.0"
map = Proj(PROJSTRING, preserve_units=False)
crs = ccrs.Stereographic(central_latitude=90.0, central_longitude=160.0)
#map = cartopy.crs.NorthPolarStereo(central_longitude=160.0)

#breakpoint()
#plotObj = romsObj.edit_mask_mesh(romsGrd.hgrid, proj=map, crs=crs, gridSubset=50)
